In [1]:
!pip install openpyxl


In [2]:
# Importar librerías
import requests
import pandas as pd
import datetime
import time
import openpyxl
from sentence_transformers import SentenceTransformer

In [3]:
posibles_params = {
    "page": 0,  # Número de página (empieza en 0)
    "pageSize": 50,  # Tamaño de página
    "order": "numeroConvocatoria",  # Campo por el que ordenar
    "direccion": "asc",  # Sentido de la ordenación: 'asc' o 'desc'
    "vpd": "GE",  # Identificador del portal
    "descripcion": "Resolución",  # Texto a buscar en el título o descripción
    "descripcionTipoBusqueda": 0,  # 0: frase exacta, 1: todas las palabras, 2: alguna palabra
    "numeroConvocatoria": "376046",  # Código BDNS a buscar
    "mrr": False,  # Mecanismo de recuperación y resiliencia
    "fechaDesde": "18/12/2017",  # Fecha de inicio (dd/mm/yyyy)
    "fechaHasta": "18/12/2017",  # Fecha de fin (dd/mm/yyyy)
    "tipoAdministracion": "C",  # 'C', 'A', 'L', 'O'
    "organos": ["713", "4730"],  # Lista de identificadores de órganos administrativos
    "regiones": [3, 50],  # Lista de identificadores de regiones
    "tiposBeneficiario": [3],  # Lista de identificadores de tipos de beneficiarios
    "instrumentos": [1],  # Lista de identificadores de instrumentos de ayuda
    "finalidad": 11,  # Identificador de la finalidad de la política de gasto
    "ayudaEstado": "SA.45221"  # Código de ayuda de estado
}

In [4]:
# Configuración de parámetros
base_url = "https://www.pap.hacienda.gob.es/bdnstrans/api/convocatorias/busqueda"
vpd = "GE"  # Identificador del portal, según la docu
page_size = 25
max_paginas = 3  # Puedes aumentar si quieres más resultados

resultados = []


# 2. Probar la API con parámetros y cabecera Accept: application/json
params = {
    "vpd": vpd,
    "page": 0,
    "pageSize": page_size
}
headers = {"Accept": "application/json"}

# Realizar una primera solicitud para verificar el Content-Type
try:
    r2 = requests.get(base_url, params=params, headers=headers)
    r2.raise_for_status() # Lanza una excepción para errores HTTP (4xx o 5xx)
except requests.exceptions.RequestException as e:
    print(f"❌ Error al conectar con la API o respuesta inicial: {e}")
    exit() # Salir si la conexión inicial falla

# 3. Si la respuesta es JSON, continuar con la descarga paginada
if "application/json" in r2.headers.get("Content-Type", ""):
    print("✅ La API responde con JSON. Descargando datos paginados...")
    for pagina in range(0, max_paginas):
        print(f"📄 Cargando página {pagina}...")
        params["page"] = pagina
        try:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()
            data = response.json()
            convocatorias = data.get("convocatorias", data.get("content", []))  # content es común en APIs paginadas
            if not convocatorias:
                print("✅ No hay más datos.")
                break
            resultados.extend(convocatorias)
            time.sleep(0.5)  # para evitar sobrecargar la API
        except Exception as e:
            print(f"❌ Error en la página {pagina}: {e}")
            break
    # Convertir a DataFrame y mostrar
    df = pd.DataFrame(resultados)
    print("Columnas disponibles:", df.columns.tolist())
    # Mostrar las primeras columnas si existen
    cols = [c for c in ["id", "titulo", "organoConvocante", "fechaPublicacion"] if c in df.columns]
else:
    print("❌ La API no responde con JSON. Revisa los parámetros, la URL o si la API está disponible.")

✅ La API responde con JSON. Descargando datos paginados...
📄 Cargando página 0...
📄 Cargando página 1...
📄 Cargando página 2...
Columnas disponibles: ['id', 'mrr', 'numeroConvocatoria', 'descripcion', 'descripcionLeng', 'fechaRecepcion', 'nivel1', 'nivel2', 'nivel3', 'codigoInvente']


In [ ]:
df.head(10)  # Mostrar las primeras filas del DataFrame

# Guardar en un archivo Excel
output_file = "data/listado_convocatorias.xlsx"
df.to_excel(output_file, index=False)

In [6]:
df.head(10)  # Mostrar las primeras filas del DataFrame

,id,mrr,numeroConvocatoria,descripcion,descripcionLeng,fechaRecepcion,nivel1,nivel2,nivel3,codigoInvente
0,1044492,False,842931,2023 CONVENIO SUBVENCION FUNDACION INGENIERIA ...,None,2025-06-29,SANTO DOMINGO DE LA CALZADA,AYUNTAMIENTO DE SANTO DOMINGO DE LA CALZADA,None,None
1,1044491,False,842930,SUBVENCIÓN NOMINATIVA CONVENIO CON FUNDACIÓN P...,None,2025-06-29,DIPUTACIÓN PROV. DE CÓRDOBA,INSTITUTO PROVINCIAL DE BIENESTAR SOCIAL,None,INV00004456
2,1044490,False,842929,Subvención Nominativa Museo Picasso Málaga 2025,None,2025-06-28,ANDALUCÍA,CONSEJERÍA DE CULTURA Y DEPORTE,None,None
3,1044489,False,842928,"Subvención nominativa 2025, Ayto Algeciras enc...",None,2025-06-28,ANDALUCÍA,CONSEJERÍA DE CULTURA Y DEPORTE,None,None
4,1044488,False,842927,Subvención nominativa 2025 Asociación Andaluza...,None,2025-06-28,ANDALUCÍA,CONSEJERÍA DE CULTURA Y DEPORTE,None,None
5,1044487,False,842926,Subvención nominativa 2025 Centro UNED Huelva,None,2025-06-28,ANDALUCÍA,"CONSEJERÍA DE UNIVERSIDAD, INVESTIGACIÓN E INN...",None,None
6,1044486,False,842925,Subvención nominativa 2025 Centro UNED Córdoba,None,2025-06-28,ANDALUCÍA,"CONSEJERÍA DE UNIVERSIDAD, INVESTIGACIÓN E INN...",None,None
7,1044485,False,842924,"Subvención Nominativa 2025,Fundación Tres Cult...",None,2025-06-28,ANDALUCÍA,CONSEJERÍA DE TURISMO Y ANDALUCÍA EXTERIOR,None,None
8,1044484,False,842923,SUBVENCIÓN NOMINATIVA 2025 UNIVERSIDAD ALMERIA...,None,2025-06-28,ANDALUCÍA,"CONSEJERÍA DE UNIVERSIDAD, INVESTIGACIÓN E INN...",None,None
9,1044483,False,842922,"Subvención Nominativa 2025, CSIC-CDLC para gas...",None,2025-06-28,ANDALUCÍA,"CONSEJERÍA DE UNIVERSIDAD, INVESTIGACIÓN E INN...",None,None


In [7]:
base_url = "https://www.infosubvenciones.es/bdnstrans/api/convocatorias"
params = {
    "vpd": "GE",         # Cambia por el portal que te interese
    "numConv": "842695"   # Número de convocatoria
}
headers = {"Accept": "application/json"}

print("🔎 Consultando convocatoria por parámetros...")
r = requests.get(base_url, params=params, headers=headers)
print("Status code:", r.status_code)
print("URL final:", r.url)
print("Content-Type:", r.headers.get("Content-Type"))
print("="*60)

if "application/json" in r.headers.get("Content-Type", ""):
    data = r.json()
    # Si la respuesta es una lista, conviértela directamente
    if isinstance(data, list):
        convocatoria = pd.DataFrame(data)
    # Si es un dict, conviértelo en DataFrame de una fila
    elif isinstance(data, dict):
        convocatoria = pd.DataFrame([data])
    else:
        print("Respuesta inesperada:", data)
        convocatoria = pd.DataFrame()
    print("Columnas disponibles:",convocatoria.columns.tolist())
else:
    print("❌ La API no responde con JSON. Revisa los parámetros, la URL o si la API está disponible.")
    convocatoria = pd.DataFrame()

🔎 Consultando convocatoria por parámetros...
Status code: 200
URL final: https://www.infosubvenciones.es/bdnstrans/api/convocatorias?vpd=GE&numConv=842695
Content-Type: application/json
Columnas disponibles: ['id', 'organo', 'sedeElectronica', 'codigoBDNS', 'fechaRecepcion', 'instrumentos', 'tipoConvocatoria', 'presupuestoTotal', 'mrr', 'descripcion', 'descripcionLeng', 'tiposBeneficiarios', 'sectores', 'regiones', 'descripcionFinalidad', 'descripcionBasesReguladoras', 'urlBasesReguladoras', 'sePublicaDiarioOficial', 'abierto', 'fechaInicioSolicitud', 'fechaFinSolicitud', 'textInicio', 'textFin', 'ayudaEstado', 'urlAyudaEstado', 'fondos', 'reglamento', 'objetivos', 'sectoresProductos', 'documentos', 'anuncios', 'advertencia']


In [8]:
convocatoria

,id,organo,sedeElectronica,codigoBDNS,fechaRecepcion,instrumentos,tipoConvocatoria,presupuestoTotal,mrr,descripcion,...,textFin,ayudaEstado,urlAyudaEstado,fondos,reglamento,objetivos,sectoresProductos,documentos,anuncios,advertencia
0,1044256,"{'nivel1': 'ANDALUCÍA', 'nivel2': 'CONSEJERÍA ...",None,842695,2025-06-26,[{'descripcion': 'SUBVENCIÓN Y ENTREGA DINERAR...,Concesión directa - instrumental,3208705.97,False,"Real Decreto 905/2022, de 25 de octubre, por e...",...,None,None,None,[{'descripcion': 'FEAGA - FONDO EUROPEO AGRÍCO...,None,[],[],"[{'id': 1286483, 'descripcion': 'Texto en cast...",[],La reutilización de los datos del Sistema Naci...


In [9]:
# Ejemplo de respuesta de una convocatoria
convocatoria

# Guardar la convocatoria en un archivo Excel
convocatoria_file = "convocatoria_842695.xlsx"
convocatoria.to_excel(convocatoria_file, index=False)

In [10]:
import os # <-- Añade esta línea

# Supón que ya tienes el DataFrame 'convocatoria' y quieres descargar todos los documentos
docs = convocatoria.iloc[0]['documentos']  # Si solo hay una convocatoria

# Carpeta donde guardar los documentos
os.makedirs("documentos_convocatoria", exist_ok=True)

for doc in docs:
    id_doc = doc['id']
    nombre = doc.get('nombreFic', f"documento_{id_doc}.pdf")
    url = f"https://www.infosubvenciones.es/bdnstrans/api/convocatorias/documentos?idDocumento={id_doc}"
    print(f"Descargando {nombre} ...")
    resp = requests.get(url)
    if resp.status_code == 200:
        with open(os.path.join("documentos_convocatoria", nombre), "wb") as f:
            f.write(resp.content)
        print(f"✅ Guardado: {nombre}")
    else:
        print(f"❌ Error al descargar {nombre} (status {resp.status_code})")

Descargando TextoConvocatoria.pdf ...
✅ Guardado: TextoConvocatoria.pdf


In [11]:
pip install transformers accelerate datasets peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
!pip install pdfplumber
import pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 103.5 MB/s eta 0:00:00


In [ ]:
pdf_folder = "data/documentos_convocatoria" # Asegúrate de que esta carpeta exista y contenga tus PDFs
output_txt_file = "data/TextoConvocatoria.txt" # El archivo de texto unificado

all_text = []

# Recorrer todos los archivos en la carpeta de PDFs
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):
        filepath = os.path.join(pdf_folder, filename)
        print(f"Extrayendo texto de: {filename}")
        try:
            with pdfplumber.open(filepath) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text: # Asegurarse de que se extrajo algo de texto
                        all_text.append(text)
        except Exception as e:
            print(f"Error al procesar {filename}: {e}")

# Unir todo el texto extraído en una sola cadena
unified_text = "\n".join(all_text)

# Guardar el texto unificado en un archivo .txt
with open(output_txt_file, "w", encoding="utf-8") as f:
    f.write(unified_text)

print(f"\nTexto de {len(os.listdir(pdf_folder))} PDFs extraído y guardado en '{output_txt_file}'")


Extrayendo texto de: TextoConvocatoria.pdf

Texto de 1 PDFs extraído y guardado en 'TextoConvocatoria.txt'


In [14]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch
from trl import SFTTrainer
import gc # Para liberar memoria
model_output_dir = "./modelo_convocatoria_fine_tuned" # Directorio para guardar el modelo entrenado
training_results_dir = "./resultados_entrenamiento" # Directorio para logs de entrenamiento

# --- 3. Cargar y procesar los datos para el Dataset ---
print("\n--- Paso 3: Cargando y procesando datos para el Dataset ---")

# Leer el contenido del documento TXT generado
with open(output_txt_file, "r", encoding="utf-8") as f:
    texto_para_entrenar = f.read()

# Crear un DataFrame simple para el dataset
data = {"text": [texto_para_entrenar]}
df = pd.DataFrame(data)

# Convertir el DataFrame a un objeto Dataset de Hugging Face
dataset = Dataset.from_pandas(df)

print("Dataset creado:")
print(dataset)
print("Ejemplo de texto en el dataset (primeros 200 caracteres):")
print(dataset[0]["text"][:200])




--- Paso 3: Cargando y procesando datos para el Dataset ---
Dataset creado:
Dataset({
    features: ['text'],
    num_rows: 1
})
Ejemplo de texto en el dataset (primeros 200 caracteres):
BOLETÍN OFICIAL DEL ESTADO
Núm. 257 Miércoles 26 de octubre de 2022 Sec. I. Pág. 145900
I. DISPOSICIONES GENERALES
MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN
17475 Real Decreto 905/2022, de 25 de


In [15]:
model_id_llm = "microsoft/phi-2"
model_id_embeddings = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" # Modelo para embeddings


# --- 3. Dividir el texto en trozos (chunks) ---
print("\n--- Paso 3: Dividiendo el texto en trozos (chunks) ---")
# Una función simple para dividir texto
def split_text_into_chunks(text, max_chunk_size=500, overlap_size=50):
    chunks = []
    current_pos = 0
    while current_pos < len(text):
        end_pos = min(current_pos + max_chunk_size, len(text))
        chunk = text[current_pos:end_pos]
        chunks.append(chunk)
        current_pos += max_chunk_size - overlap_size # Mover hacia adelante con solapamiento
        if current_pos >= len(text) - overlap_size: # Para asegurar el último trozo
            break
    return chunks

text_chunks = split_text_into_chunks(unified_text, max_chunk_size=500, overlap_size=50)
print(f"Texto dividido en {len(text_chunks)} trozos.")
print(f"Primer trozo de ejemplo: {text_chunks[0][:150]}...")


--- Paso 3: Dividiendo el texto en trozos (chunks) ---
Texto dividido en 815 trozos.
Primer trozo de ejemplo: BOLETÍN OFICIAL DEL ESTADO
Núm. 257 Miércoles 26 de octubre de 2022 Sec. I. Pág. 145900
I. DISPOSICIONES GENERALES
MINISTERIO DE AGRICULTURA, PESCA Y ...


In [16]:
# --- 4. Generar y almacenar embeddings (Base de datos vectorial en memoria) ---
device_embeddings = "cuda" if torch.cuda.is_available() else "cpu"

# Cargar el modelo de embeddings
embedding_model = SentenceTransformer(model_id_embeddings, device=device_embeddings)

# Generar embeddings para cada trozo
chunk_embeddings = embedding_model.encode(text_chunks, convert_to_tensor=True, show_progress_bar=True)
print(f"Embeddings generados. Forma de los embeddings: {chunk_embeddings.shape}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Embeddings generados. Forma de los embeddings: torch.Size([815, 384])


In [17]:

# --- 5. Cargar el LLM para inferencia (microsoft/phi-2) ---

tokenizer_llm = AutoTokenizer.from_pretrained(model_id_llm, trust_remote_code=True)

model_llm = AutoModelForCausalLM.from_pretrained(
    model_id_llm,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float16,
    device_map="auto",
    trust_remote_code=True,
)
model_llm.eval() # Poner el modelo en modo evaluación

# Asegurar que el tokenizer tenga un pad_token, necesario para la generación.
if tokenizer_llm.pad_token is None:
    tokenizer_llm.pad_token = tokenizer_llm.eos_token
tokenizer_llm.padding_side = 'left' # Para Phi-2, 'left' suele ser mejor para la generación.

print("LLM cargado y listo para inferencia.")



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

LLM cargado y listo para inferencia.


In [18]:

# --- 6. Función de Preguntas y Respuestas (RAG) ---

def preguntar_al_modelo_rag(pregunta_usuario, top_k=3, max_new_tokens=200, temperature=0.7):
    # a. Generar embedding para la pregunta del usuario
    local_embedding_model = SentenceTransformer(model_id_embeddings, device=device_embeddings)
    pregunta_embedding = local_embedding_model.encode(pregunta_usuario, convert_to_tensor=True).to(device_embeddings)

    # b. Buscar los trozos más relevantes (similitud del coseno)
    similarities = torch.nn.functional.cosine_similarity(pregunta_embedding.unsqueeze(0), chunk_embeddings)
    top_k_indices = torch.topk(similarities, top_k).indices.tolist()

    relevant_chunks = [text_chunks[i] for i in top_k_indices]
    context = "\n\n".join(relevant_chunks)

    del local_embedding_model
    if device_embeddings == "cuda":
        torch.cuda.empty_cache()
    gc.collect()

    # c. Formular el prompt para el LLM con el contexto
    # Phi-2 es un modelo de base, no chat. Un prompt simple de instrucción funciona bien:
    prompt = f"""Instrucción: Basándote ÚNICAMENTE en el siguiente texto de contexto, responde a la pregunta. Si la información no está en el contexto, simplemente di que no tienes suficiente información.

Contexto:
{context}

Pregunta: {pregunta_usuario}
Respuesta: """

    # d. Generar la respuesta usando el LLM
    input_ids = tokenizer_llm(prompt, return_tensors="pt", return_attention_mask=False).to(model_llm.device) # return_attention_mask=False para Phi-2

    with torch.no_grad():
        output = model_llm.generate(
            **input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            pad_token_id=tokenizer_llm.pad_token_id, # Usar pad_token_id
            eos_token_id=tokenizer_llm.eos_token_id # Asegúrate de que el token EOS es correcto
        )

    response = tokenizer_llm.decode(output[0], skip_special_tokens=False)

    # Limpiar el prompt de la respuesta generada.
    # Phi-2 simplemente continuará el texto, así que la respuesta empezará después del "Respuesta: "
    if "Respuesta: " in response:
        cleaned_response = response.split("Respuesta: ", 1)[1].strip()
        # Eliminar cualquier token de fin de secuencia que el modelo pueda generar
        cleaned_response = cleaned_response.replace("<|endoftext|>", "").strip()
        cleaned_response = cleaned_response.split("Instrucción:", 1)[0].strip() # Quitar repetición si genera mucho
    else:
        cleaned_response = response # Fallback si el formato no coincide exactamente

    return cleaned_response

In [19]:

# --- Interacción con el usuario ---
print("\n--- ¡Sistema de preguntas y respuestas RAG listo con microsoft/phi-2! ---")
print("Puedes empezar a hacer preguntas sobre tus documentos de convocatoria.")
print("Escribe 'salir' para terminar.")

while True:
    user_question = input("\nTu pregunta: ")
    if user_question.lower() == 'salir':
        break

    print("Buscando y generando respuesta...")
    answer = preguntar_al_modelo_rag(user_question)
    print(f"Respuesta del asistente: {answer}")

print("\nFin del programa. ¡Hasta pronto!")


--- ¡Sistema de preguntas y respuestas RAG listo con microsoft/phi-2! ---
Puedes empezar a hacer preguntas sobre tus documentos de convocatoria.
Escribe 'salir' para terminar.

Tu pregunta: Que ayudas hay para el sector viñícola
Buscando y generando respuesta...


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Respuesta del asistente: Para el sector viñícola, hay varias formas de apoyo asegurando que el mercado se mantenga equilibrado y que los productores se recuperan de sus recursos. Uno de estos formas es el Programa de apoyo al sector vitivinícola (PASVE), que es una inversión de la UE para el sector agrario. El PASVE tiene como objetivo promover el desarrollo de la viticultura y la vinificación, así como la productividad y la innovación. El PASVE tiene una budget de 1.2 millones de euros, y se distribuye de manera equitativa a las regiones de España. El PASVE se puede enviar una solicitud de ay

Tu pregunta: salir

Fin del programa. ¡Hasta pronto!
